## Validación de datos

Vamos a hacer ahora una clase práctica de navegacion en lso datos, compararemos dos dfs donde uno es un espejo del otro con errorcillos para intentar encontrar esos errores, esas líneas diferentes que luego nos conducirian en el código a buscar la sentencia corrupta

#### Importamos las librerías necesarias para hacer el análisis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Cargamos el dataset

In [ ]:
df_test = pd.read_excel('Validacion_datos_example.xlsx', sheet_name="Test")

In [ ]:
df_output = pd.read_excel('Validacion_datos_example.xlsx', sheet_name="Output")

#### Asegurarse de que test y output tienen los mismos formatos

In [ ]:
# Empezamos viendo las primeras filas de los dfs para ver a simple vista si tienen el mismo aspecto
df_test.head()

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
0,SEAT,Toledo,950,Diésel,2000.0,227000.0,4.0,Manual
1,CITROEN,C1,6200,Gasolina,2017.0,50071.0,5.0,Manual
2,FORD,Transit Connect,7851,Diésel,2016.0,103000.0,4.0,Manual
3,VOLKSWAGEN,Caravelle,19426,Diésel,2014.0,120000.0,4.0,Manual
4,FORD,Transit,22850,Diésel,2017.0,107000.0,2.0,Manual


In [ ]:
df_output.head()

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
0,SEAT,Toledo,950,Diésel,2000.0,227000.0,4.0,Manual
1,CITROEN,C1,6200,Gasolina,2017.0,50071.0,5.0,Manual
2,FORD,Transit Connect,7851,Diésel,2016.0,103000.0,4.0,Manual
3,VOLKSWAGEN,Caravelle,19426,Diésel,2014.0,120000.0,4.0,Manual
4,FORD,Transit,22850,Diésel,2017.0,107000.0,2.0,Manual


#### Analizar las columnas categóricas por valores únicos en test vs output

In [ ]:
# miramos todos los valores únicos de las columnas categóricas
# Lo primero es saber cuales son las cátegoricas, podemos verlo a simple vista , o filtrar el df por el tipo de columna
print(df_test.select_dtypes(include=[object]).columns == df_output.select_dtypes(include=[object]).columns)
print(df_test.select_dtypes(include=[object]).columns)
print(df_output.select_dtypes(include=[object]).columns)

[ True  True  True  True]
Index(['Marca', 'Modelo', 'Tipo de combustible', 'Tipo de cambio'], dtype='object')
Index(['Marca', 'Modelo', 'Tipo de combustible', 'Tipo de cambio'], dtype='object')


In [ ]:
dict_test = {}
for col in df_test.select_dtypes(include=[object]):
    dict_test[col]= df_test[col].unique()
dict_output = {}
for col in df_output.select_dtypes(include=[object]):
    dict_output[col]= df_output[col].unique()


In [ ]:
# como son iguales las columnas, ahora recorremos los diccionarios para ver las desigualdades dentro de cada columna
# imprimimos el nombre de la columna y los valores diferentes de esa columna entre el df_output y el df_test
for col in df_test.select_dtypes(include=[object]):
    print(col)
    print(list(set(dict_output["Tipo de cambio"]) - set(dict_test["Tipo de cambio"][dict_test["Tipo de cambio"]== dict_output["Tipo de cambio"]])))

Marca
[nan]
Modelo
[nan]
Tipo de combustible
[nan]
Tipo de cambio
[nan]


Vemos que las diferencias son los valores nan, eso es porque en python nan != nan, por lo que podriamos tratar los nans y luego repetir, o sencillamente ya sabemos que no hay valores unicos diferentes entre las columnas categóricas de ambos dfs

In [ ]:
# Comprobamos si el tipo de datos de las columnas es igual
df_test.dtypes == df_output.dtypes

Marca                  True
Modelo                 True
Precio                 True
Tipo de combustible    True
Año                    True
Kilómetros             True
Número de puertas      True
Tipo de cambio         True
dtype: bool

A simple vista parece el mismo df

#### Analizar las columnas numéricas en test vs output

In [ ]:
# Comprobamos tambien si el tamaño de los dfs y los nulos son iguales
df_test.info() == df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Marca                49998 non-null  object 
 1   Modelo               49995 non-null  object 
 2   Precio               50000 non-null  int64  
 3   Tipo de combustible  49954 non-null  object 
 4   Año                  49993 non-null  float64
 5   Kilómetros           49995 non-null  float64
 6   Número de puertas    49993 non-null  float64
 7   Tipo de cambio       49889 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 3.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Marca                49998 non-null  object 
 1   Modelo               49995 non-null  object 
 2   Precio           

True

Hay que fijarse pero vemos que hay menos nulls en el numero de puertas de la salida que en el de test

In [ ]:
# Comprobamos si los valores numéricos son iguales mirando sus variables estadísticas
# Si todos los valores son iguales, es probable que la salida de nuestro proceso sea correcta y dé lo que esperábamos
df_test.describe() == df_output.describe()

,Precio,Año,Kilómetros,Número de puertas
count,True,True,True,False
mean,False,True,True,False
std,False,True,True,False
min,True,True,True,True
25%,True,True,True,True
50%,True,True,True,True
75%,True,True,True,True
max,True,True,True,True


En el precio bailan la media y en la desviación estándar

In [ ]:
# Vamos a ver la media que es algo que no muestra el describe de serie
print(df_test.Precio.mean())
print(df_output.Precio.mean())
df_test.Precio.mean()==df_output.Precio.mean()

15166.27026
15166.24046


False

Algo no está bien, nos toca buscar que es, vamos a entrar en los datos poco a poco

#### Comparamos los dataframes para buscar registros difrentes en ellos, de esa manera si son pocos sacamos las diferencias

In [ ]:
# Para eso los mergeamos marcando las lineas con el indicador de si esa fila está en los dos, en la izquierda o en la derecha
join_left = df_test.merge(df_output, how='left', indicator=True)
lineas_left = join_left[join_left['_merge'] != 'both']
lineas_left

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge
42542,SEAT,Córdoba,2600,Gasolina,2008.0,167857.0,NaN,Manual,left_only
42543,FIAT,500,8499,Gasolina,2017.0,59400.0,NaN,Manual,left_only
42544,AUDI,A3,17250,Diésel,2016.0,79211.0,NaN,Manual,left_only
42552,FORD,Fiesta,7500,Diésel,2016.0,129000.0,5.0,Manual,left_only
42693,PEUGEOT,Partner,9500,Diésel,2014.0,152140.0,4.0,Manual,left_only


In [ ]:
join_rigth = df_test.merge(df_output, how='right', indicator=True)
lineas_right = join_rigth[join_rigth['_merge'] != 'both']
lineas_right

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge
44438,SEAT,Córdoba,2600,Gasolina,2008.0,167857.0,3.0,Manual,right_only
44439,FIAT,500,8499,Gasolina,2017.0,59400.0,3.0,Manual,right_only
44440,AUDI,A3,17250,Diésel,2016.0,79211.0,3.0,Manual,right_only
44448,FORD,Fiesta,18500,Diésel,2016.0,129000.0,5.0,Manual,right_only
44569,PEUGEOT,Partner,9500,Híbrido,2014.0,152140.0,4.0,Manual,right_only
44673,KIA,Stonic,4500,Gasolina,2019.0,10700.0,5.0,Manual,right_only


In [ ]:
# Y las unimos en un solo df para verlas juntas y ordenadas por las diferencias
lineas_diff = pd.concat([lineas_left, lineas_right], ignore_index=True).sort_values(by=list(lineas_left.columns),axis=0)
lineas_diff

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge
7,AUDI,A3,17250,Diésel,2016.0,79211.0,3.0,Manual,right_only
2,AUDI,A3,17250,Diésel,2016.0,79211.0,NaN,Manual,left_only
6,FIAT,500,8499,Gasolina,2017.0,59400.0,3.0,Manual,right_only
1,FIAT,500,8499,Gasolina,2017.0,59400.0,NaN,Manual,left_only
3,FORD,Fiesta,7500,Diésel,2016.0,129000.0,5.0,Manual,left_only
8,FORD,Fiesta,18500,Diésel,2016.0,129000.0,5.0,Manual,right_only
10,KIA,Stonic,4500,Gasolina,2019.0,10700.0,5.0,Manual,right_only
4,PEUGEOT,Partner,9500,Diésel,2014.0,152140.0,4.0,Manual,left_only
9,PEUGEOT,Partner,9500,Híbrido,2014.0,152140.0,4.0,Manual,right_only
5,SEAT,Córdoba,2600,Gasolina,2008.0,167857.0,3.0,Manual,right_only


Otra manera más rápida seria unir los dos dfs y quitar las lineas duplicadas, asi sólo nos quedarian las diferentes

In [ ]:
df_diff = pd.concat([df_test, df_output]).drop_duplicates(keep=False).sort_values(by=list(df_test.columns),axis=0)
df_diff

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
38468,AUDI,A3,17250,Diésel,2016.0,79211.0,3.0,Manual
38468,AUDI,A3,17250,Diésel,2016.0,79211.0,NaN,Manual
38467,FIAT,500,8499,Gasolina,2017.0,59400.0,3.0,Manual
38467,FIAT,500,8499,Gasolina,2017.0,59400.0,NaN,Manual
38476,FORD,Fiesta,7500,Diésel,2016.0,129000.0,5.0,Manual
38476,FORD,Fiesta,18500,Diésel,2016.0,129000.0,5.0,Manual
38702,KIA,Stonic,4500,Gasolina,2019.0,10700.0,5.0,Manual
38600,PEUGEOT,Partner,9500,Diésel,2014.0,152140.0,4.0,Manual
38600,PEUGEOT,Partner,9500,Híbrido,2014.0,152140.0,4.0,Manual
38466,SEAT,Córdoba,2600,Gasolina,2008.0,167857.0,3.0,Manual


Este segundo método tendria el ligerísimo inconveniente de que no sabes que registro viene de cada lado

Ahora nos tocaría filtrar las diferencias para aislar un caso determinado, luego buscar en el código dónde se crea esa línea y por último, seguirla para ver dónde se realizar la operación o cálculo erróneo.

In [ ]:
df_diff_filtered = df_diff[(df_diff["Marca"]=="AUDI") & (df_diff["Modelo"]=="A3")]
df_diff_filtered

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
38468,AUDI,A3,17250,Diésel,2016.0,79211.0,3.0,Manual
38468,AUDI,A3,17250,Diésel,2016.0,79211.0,NaN,Manual


Sea como fuere, el procedimiento para encontrar las diferencias y aislarlas es siempre el que hemos visto, iterando hasta que el resultado "output" sea justo lo que esperabamos , tengamos un test con el que compararnos o no.

Tambien existe una aproximación diferente, donde se agrupan los valores y se va profundizando progresivamente para ver buscar la línea exacta que esta mal.

Esto es más lento que comparar los dos dfs como hemos hecho antes pero por el camino, al ir paso a paso, muchas veces detectamos problemas que se explican mejor con los valores agrupados que linea a linea.

Por ejemplo si se realizan calculos agrupados por marca o por modelo , la linea quizás no nos refleje todo el problema.

Vamos a ver ahora cómo hilar muy fino, yendo de menos detalle a más para buscar valores diferentes entre los dos dataframes artesanalmente

#### Agrupamos datos y buscamos los problemas

De mayor a menor: Se agrupan los datos por los conjuntos más grandes, si no son iguales, se va aumentando la granularidad hasta el máximo posible en el test y el output

Nos quedamos con el precio para investigar más, y de categórica elegimos en primera instancia la Marca

In [ ]:
df_output_grouped = df_output[["Marca","Precio"]].groupby(['Marca']).sum().reset_index()
df_output_grouped = df_output_grouped.sort_values("Precio").reset_index(drop= True)
df_output_grouped

,Marca,Precio
0,VAZ,500
1,ARO,1000
2,IVECO-PEGASO,2000
3,DAIHATSU,3500
4,UMM,8000
...,...,...
70,PEUGEOT,37784267
71,VOLKSWAGEN,60759846
72,AUDI,71611150
73,BMW,84890573


In [ ]:
df_test_grouped = df_test[["Marca","Precio"]].groupby(['Marca']).sum().reset_index()
df_test_grouped = df_test_grouped.sort_values("Precio").reset_index(drop= True)
df_test_grouped

,Marca,Precio
0,VAZ,500
1,ARO,1000
2,IVECO-PEGASO,2000
3,DAIHATSU,3500
4,UMM,8000
...,...,...
70,PEUGEOT,37784267
71,VOLKSWAGEN,60759846
72,AUDI,71611150
73,BMW,84890573


Juntamos los dfs agrupados para buscar las marcas que no son iguales en ambos lados

In [ ]:
df3 = df_test_grouped.merge(df_output_grouped,on='Marca', how='left')
df3["diff"] = df3["Precio_x"]-df3["Precio_y"]
df3[df3["diff"]!=0]

,Marca,Precio_x,Precio_y,diff
59,KIA,16499427,16486937,12490
69,FORD,32078339,32089339,-11000


Ya sabemos que dos marcas son distintas, ahora repetimos el proceso añadiendo otra columna al group by pero filtrando por Kia

In [ ]:
df_output_filtered = df_output[df_output["Marca"]=="KIA"]
df_output_grouped2 = df_output_filtered[["Modelo","Precio"]].groupby(['Modelo']).sum().reset_index()
df_output_grouped2 = df_output_grouped2.sort_values("Precio").reset_index(drop= True)
df_output_grouped2

,Modelo,Precio
0,Sephia II,1500
1,Magentis,1800
2,Pregio,3490
3,León,3995
4,Shuma,5690
5,k2500,6500
6,Cerato,9190
7,ceed Sporty Wagon,13550
8,pro_ceed GT,16490
9,Niro Hïbrido Enchufable,29900


In [ ]:
df_test_filtered = df_test[df_test["Marca"]=="KIA"]
df_test_grouped2 = df_test_filtered[["Modelo","Precio"]].groupby(['Modelo']).sum().reset_index()
df_test_grouped2 = df_test_grouped2.sort_values("Precio").reset_index(drop= True)
df_test_grouped2

,Modelo,Precio
0,Sephia II,1500
1,Magentis,1800
2,Pregio,3490
3,León,3995
4,Shuma,5690
5,k2500,6500
6,Cerato,9190
7,ceed Sporty Wagon,13550
8,pro_ceed GT,16490
9,Niro Hïbrido Enchufable,29900


In [ ]:
df4 = df_test_grouped2.merge(df_output_grouped2,on='Modelo', how='left')
df4["diff"] = df4["Precio_x"]-df4["Precio_y"]
df4[df4["diff"]!=0]

,Modelo,Precio_x,Precio_y,diff
30,Stonic,1443489,1430999,12490


Ya sabemos que el KIA Stonic nos está devolviendo diferencias, vamos a ver que lineas son esas en los dfs, cuantas hay y con esto podriamos entender la lógica del error en nuestro código para ver porqué esa diferencia existe

In [ ]:
df_output_filtered2 = df_output[(df_output["Marca"]=="KIA") & (df_output["Modelo"]=="Stonic")]
print(df_output_filtered2.count())
df_output_filtered2

Marca                  88
Modelo                 88
Precio                 88
Tipo de combustible    88
Año                    87
Kilómetros             88
Número de puertas      88
Tipo de cambio         88
dtype: int64


,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
341,KIA,Stonic,15495,Gasolina,2018.0,30000.0,5.0,Manual
758,KIA,Stonic,16900,Gasolina,2019.0,11400.0,5.0,Manual
991,KIA,Stonic,16216,Gasolina,2019.0,13123.0,5.0,Manual
1029,KIA,Stonic,17790,Gasolina,2020.0,5.0,5.0,Manual
1282,KIA,Stonic,15500,Gasolina,2019.0,12000.0,5.0,Manual
...,...,...,...,...,...,...,...,...
46595,KIA,Stonic,16990,Gasolina,2020.0,0.0,5.0,Manual
47024,KIA,Stonic,16100,Gasolina,2019.0,16000.0,5.0,Manual
47226,KIA,Stonic,17390,Gasolina,2020.0,5.0,5.0,Manual
48955,KIA,Stonic,16300,Gasolina,2019.0,18500.0,5.0,Manual


Todavia hay muchos registros asi que tendríamos que seguir iterando sobre otras columnas como el año , el tipo de combustible, etc... hasta encontrar los registros que nos están haciendo el lío y despues entender por qué nos lo están haciendo...

In [ ]:
df_output_filtered3 = df_output[(df_output["Marca"]=="KIA") & (df_output["Modelo"]=="Stonic")]
df_output_grouped3 = df_output_filtered3[["Kilómetros","Precio"]].groupby(['Kilómetros']).sum().reset_index()
df_output_grouped3 = df_output_grouped3.sort_values("Precio").reset_index(drop= True)

df_test_filtered3 = df_test[(df_test["Marca"]=="KIA") & (df_test["Modelo"]=="Stonic")]
df_test_grouped3 = df_test_filtered3[["Kilómetros","Precio"]].groupby(['Kilómetros']).sum().reset_index()
df_test_grouped3 = df_test_grouped3.sort_values("Precio").reset_index(drop= True)


df4 = df_test_grouped3.merge(df_output_grouped3,on='Kilómetros', how='left')
df4["diff"] = df4["Precio_x"]-df4["Precio_y"]
df4[df4["diff"]!=0]

,Kilómetros,Precio_x,Precio_y,diff
53,10700.0,33980,21490,12490


In [ ]:
df_output_filtered3 = df_output[(df_output["Marca"]=="KIA") & (df_output["Modelo"]=="Stonic") & (df_output["Kilómetros"]==10700)]
df_output_filtered3

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
38702,KIA,Stonic,4500,Gasolina,2019.0,10700.0,5.0,Manual
40186,KIA,Stonic,16990,Gasolina,2019.0,10700.0,5.0,Manual


In [ ]:
df_test_filtered3 = df_test[(df_test["Marca"]=="KIA") & (df_test["Modelo"]=="Stonic") & (df_test["Kilómetros"]==10700)]
df_test_filtered3

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
38702,KIA,Stonic,16990,Gasolina,2019.0,10700.0,5.0,Manual
40186,KIA,Stonic,16990,Gasolina,2019.0,10700.0,5.0,Manual


#### ¡Eureka!

No sólo hemos encontrado la diferencia, sino que además es algo que NO habiamos visto de la otra manera, y es que si nos fijamos, hay una línea distinta, la 38702, pero es que además esa línea habia quedado enmascarada con el procedimiento de buscar las diferencias en el df completo por estar duplicada en el df de test, de forma que si existina en ambos dfs, solo que en el de la salida estaba una sóla vez.

## Ejercicio

Comparar la hoja Test vs hoja Exercise para buscar las diferencias, sabemos que nuestro proceso saca valores que no estan siempre bien pero no sabemos porqué, ni cuale son, encuentra porqué y despues encuentra la lógica del error.

In [ ]:
df_test = pd.read_excel('Validacion_datos_example.xlsx', sheet_name="Test")

In [ ]:
df_exer = pd.read_excel('Validacion_datos_example.xlsx', sheet_name="Exercise")

In [ ]:
df_diff = pd.concat([df_test, df_exer]).drop_duplicates(keep=False).sort_values(by=list(df_test.columns),axis=0)
df_diff

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
44455,FIAT,500,12900,Híbrido,2020.0,350.0,3.0,Manual
16425,FIAT,500,14500,Híbrido,2020.0,6.0,3.0,Manual
36467,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual
49422,FIAT,500,14990,Híbrido,2020.0,10.0,3.0,Manual
18137,FIAT,500,15000,Híbrido,2020.0,10.0,3.0,Manual
...,...,...,...,...,...,...,...,...
14347,SUZUKI,Vitara,48980,Híbrido,2020.0,14.0,5.0,Manual
11624,SUZUKI,Vitara,50900,Híbrido,2020.0,0.0,5.0,Manual
12107,SUZUKI,Vitara,51880,Híbrido,2020.0,0.0,5.0,Manual
13186,SUZUKI,Vitara,53180,Híbrido,2020.0,4.0,5.0,Manual


In [ ]:
join_left = df_test.merge(df_exer, how='left', indicator=True)
lineas_left = join_left[join_left['_merge'] != 'both']

join_rigth = df_test.merge(df_exer, how='right', indicator=True)
lineas_right = join_rigth[join_rigth['_merge'] != 'both']

lineas_diff = pd.concat([lineas_left, lineas_right], ignore_index=True).sort_values(by=list(lineas_left.columns),axis=0)
lineas_diff

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge
83,FIAT,500,12900,Híbrido,2020.0,350.0,3.0,Manual,left_only
33,FIAT,500,14500,Híbrido,2020.0,6.0,3.0,Manual,left_only
68,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual,left_only
95,FIAT,500,14990,Híbrido,2020.0,10.0,3.0,Manual,left_only
38,FIAT,500,15000,Híbrido,2020.0,10.0,3.0,Manual,left_only
...,...,...,...,...,...,...,...,...,...
125,SUZUKI,Vitara,48980,Híbrido,2020.0,14.0,5.0,Manual,right_only
118,SUZUKI,Vitara,50900,Híbrido,2020.0,0.0,5.0,Manual,right_only
119,SUZUKI,Vitara,51880,Híbrido,2020.0,0.0,5.0,Manual,right_only
121,SUZUKI,Vitara,53180,Híbrido,2020.0,4.0,5.0,Manual,right_only


In [ ]:
dict_diff = {}
for col in lineas_diff:
    dict_diff[col]= lineas_diff[col].unique()

In [ ]:
dict_diff

{'Marca': array(['FIAT', 'FORD', 'HYUNDAI', 'KIA', 'SUZUKI'], dtype=object),
 'Modelo': array(['500', '500C', 'Fiesta', 'Focus', 'Kuga', 'Puma', 'Transit Custom',
        'TUCSON', 'Niro', 'Sportage', 'Ignis', 'SCross', 'Swift', 'Vitara'],
       dtype=object),
 'Precio': array([12900, 14500, 14700, 14990, 15000, 16240, 16400, 17200, 25800,
        29000, 29400, 29980, 30000, 32480, 32800, 34400, 15290, 18360,
        18490, 30580, 36720, 36980, 16450, 16490, 16900, 16990, 32900,
        32980, 33800, 33980, 21445, 22499, 22800, 27290, 42890, 44998,
        45600, 54580, 27600, 28690, 29500, 55200, 57380, 59000, 18500,
        19500, 19700, 20800, 20900, 20950, 21490, 21590, 21600, 21700,
        21900, 22400, 23500, 23990, 25200, 25490, 26010, 27500, 37000,
        39000, 39400, 41600, 41800, 41900, 42980, 43180, 43200, 43400,
        43800, 44800, 47000, 47980, 50400, 50980, 51600, 52020, 55000,
        28900, 30900, 57800, 61800, 23890, 24490, 24500, 24990, 25400,
        25500, 266

Ojo!, porque aqui ya vemos cosas raras, como que todos los que han cambiado tienen Tipo de combustible Híbridos Tipo de cambio manual.

In [ ]:
print(lineas_diff["Marca"].count()/2)
print(df_exer[(df_exer["Tipo de cambio"]=="Manual") & (df_exer["Tipo de combustible"]=="Híbrido")]["Marca"].count())

96.0
160


 Nos da una pista, sin embargo aun se nos escapa algo porque ese filtro nos devovlería más lineas que las diferentes y además no entendemos todavia bien qué es lo que hace mal.

 Vamos a coger y buscar relacion entre las lineas modificadas y las originales para intentar encontrar que pasa en el cálculo

In [ ]:
df_final = lineas_diff[lineas_diff["_merge"]=="left_only"].merge(lineas_diff[lineas_diff["_merge"]=="right_only"],
                                                                 on=["Marca","Modelo","Tipo de combustible","Año","Kilómetros","Número de puertas", "Tipo de cambio"],
                                                                 how='left')
df_final["relation1"] = df_final["Precio_x"]-df_final["Precio_y"]
df_final["relation2"] = df_final["Precio_x"]/df_final["Precio_y"]
df_final[(df_final["relation1"]!=0) | (df_final["relation2"]!=1)]



,Marca,Modelo,Precio_x,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge_x,Precio_y,_merge_y,relation1,relation2
0,FIAT,500,12900,Híbrido,2020.0,350.0,3.0,Manual,left_only,25800,right_only,-12900,0.500000
1,FIAT,500,14500,Híbrido,2020.0,6.0,3.0,Manual,left_only,29000,right_only,-14500,0.500000
2,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual,left_only,29400,right_only,-14700,0.500000
3,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual,left_only,32480,right_only,-17780,0.452586
4,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual,left_only,32800,right_only,-18100,0.448171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,SUZUKI,Vitara,27200,Híbrido,2020.0,0.0,5.0,Manual,left_only,46600,right_only,-19400,0.583691
236,SUZUKI,Vitara,27200,Híbrido,2020.0,0.0,5.0,Manual,left_only,47620,right_only,-20420,0.571189
237,SUZUKI,Vitara,27200,Híbrido,2020.0,0.0,5.0,Manual,left_only,50900,right_only,-23700,0.534381
238,SUZUKI,Vitara,27200,Híbrido,2020.0,0.0,5.0,Manual,left_only,51880,right_only,-24680,0.524287


A veces existe una relation2 de 0,5 y a veces no, eso nos indica que los valores se están duplicando, pero además nos dice que no siempre.

Esto está mal, sabrias decir porque no siempre es 0,5?

Efectivamente!!, el problema es que hemos cruzado por todos los campos peero resulta que tenemos varios registros iguales con la misma conbinacion de campos , por lo que el cruce nos está devolviendo valores que no existen realmente, se produce un producto cartesiano parcial sin querer

Vamos a hacer otra vez la diferencia pero sobre los campos agrupados para ver el total vs total

In [ ]:
df_diff_orig = lineas_diff[lineas_diff["_merge"]=="left_only"]
df_diff_ex = lineas_diff[lineas_diff["_merge"]=="right_only"]
columns_group = ["Marca","Modelo","Tipo de combustible","Año","Kilómetros","Número de puertas", "Tipo de cambio"]

df_diff_orig_agrup = df_diff_orig.groupby(columns_group).sum().reset_index()
df_diff_ex_agrup = df_diff_ex.groupby(columns_group).sum().reset_index()


df_final_agrup = df_diff_orig_agrup.merge(df_diff_ex_agrup,
                                          on=columns_group,
                                          how='left')
df_final_agrup["relation1"] = df_final_agrup["Precio_x"]-df_final_agrup["Precio_y"]
df_final_agrup["relation2"] = df_final_agrup["Precio_x"]/df_final_agrup["Precio_y"]
df_final_agrup[(df_final_agrup["relation1"]!=0) | (df_final_agrup["relation2"]!=1)]

,Marca,Modelo,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,Precio_x,Precio_y,relation1,relation2
0,FIAT,500,Híbrido,2020.0,1.0,3.0,Manual,64540,129080,-64540,0.5
1,FIAT,500,Híbrido,2020.0,6.0,3.0,Manual,14500,29000,-14500,0.5
2,FIAT,500,Híbrido,2020.0,10.0,3.0,Manual,29990,59980,-29990,0.5
3,FIAT,500,Híbrido,2020.0,350.0,3.0,Manual,12900,25800,-12900,0.5
4,FIAT,500C,Híbrido,2020.0,1.0,2.0,Manual,18360,36720,-18360,0.5
5,FIAT,500C,Híbrido,2020.0,300.0,2.0,Manual,15290,30580,-15290,0.5
6,FIAT,500C,Híbrido,2021.0,1.0,2.0,Manual,18490,36980,-18490,0.5
7,FORD,Fiesta,Híbrido,2020.0,1.0,5.0,Manual,49340,98680,-49340,0.5
8,FORD,Fiesta,Híbrido,2020.0,10.0,5.0,Manual,50880,101760,-50880,0.5
9,FORD,Focus,Híbrido,2020.0,0.0,5.0,Manual,48735,97470,-48735,0.5


#### ¡Eureka!

Todas las relaciones2 son 0,5, ahora sí!!!, ya hemos descubierto que estaba sucediendo, y es que duplica los valores de algunos registros!.

Además, ya sabemos que esos registros son los hibridos manuales , pero aun os queda afinal algo más, proque son los hibridos manuales y algo más , vamos a ver si descubrimos que condiciones nos faltan por encontrar.

Cogemos un modelo del df original y buscamos los que se han modificado y los que no , es la diferencia entre los que si y los que no la que nos va a dar la pista final

In [ ]:
lineas_original_filtered_raw = df_test[(df_test["Marca"]=="FIAT")
                                   & (df_test["Modelo"]=="500")
                                  & (df_test["Tipo de combustible"]=="Híbrido")
                                  & (df_test["Tipo de cambio"]=="Manual")]
lineas_original_filtered_raw

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
15956,FIAT,500,16240,Híbrido,2020.0,1.0,3.0,Manual
16425,FIAT,500,14500,Híbrido,2020.0,6.0,3.0,Manual
18137,FIAT,500,15000,Híbrido,2020.0,10.0,3.0,Manual
23405,FIAT,500,13950,Híbrido,2020.0,1200.0,3.0,Manual
24541,FIAT,500,12995,Híbrido,2020.0,4799.0,3.0,Manual
28970,FIAT,500,17200,Híbrido,2020.0,1.0,3.0,Manual
35137,FIAT,500,13600,Híbrido,2020.0,3000.0,3.0,Manual
36467,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual
38987,FIAT,500,16400,Híbrido,2020.0,1.0,3.0,Manual
44455,FIAT,500,12900,Híbrido,2020.0,350.0,3.0,Manual


In [ ]:
lineas_diff_filtered = lineas_diff[(lineas_diff["Marca"]=="FIAT") & (lineas_diff["Modelo"]=="500")& (lineas_diff["_merge"]=="left_only")]
lineas_diff_filtered

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio,_merge
83,FIAT,500,12900,Híbrido,2020.0,350.0,3.0,Manual,left_only
33,FIAT,500,14500,Híbrido,2020.0,6.0,3.0,Manual,left_only
68,FIAT,500,14700,Híbrido,2020.0,1.0,3.0,Manual,left_only
95,FIAT,500,14990,Híbrido,2020.0,10.0,3.0,Manual,left_only
38,FIAT,500,15000,Híbrido,2020.0,10.0,3.0,Manual,left_only
31,FIAT,500,16240,Híbrido,2020.0,1.0,3.0,Manual,left_only
74,FIAT,500,16400,Híbrido,2020.0,1.0,3.0,Manual,left_only
55,FIAT,500,17200,Híbrido,2020.0,1.0,3.0,Manual,left_only


Nos quedamos con los que son diferentes y buscamos la diferencia entre el lineas_diff_filtered vs lineas_original_filtered

In [ ]:
lineas_original_filtered = pd.concat([lineas_diff_filtered.iloc[:, :-1], lineas_original_filtered_raw]).drop_duplicates(keep=False)
lineas_original_filtered

,Marca,Modelo,Precio,Tipo de combustible,Año,Kilómetros,Número de puertas,Tipo de cambio
23405,FIAT,500,13950,Híbrido,2020.0,1200.0,3.0,Manual
24541,FIAT,500,12995,Híbrido,2020.0,4799.0,3.0,Manual
35137,FIAT,500,13600,Híbrido,2020.0,3000.0,3.0,Manual
47898,FIAT,500,12790,Híbrido,2020.0,2208.0,3.0,Manual


Vemos que ni la marca, ni el modelo , ni el tipo de combustible, ni el año , ni el numero de puertas , ni el tipo de cambio nos estan afectando, eso solo nos deja el precio y los kilómetros. Y como el precio es lo que se ha modificado realmente, sólo puede ser algo que sucede en los kilómetros.

¿Sabrías ver que es?

#### Efectivamente!

Las lineas que se modifican tienen kms pequeños o dicho de otro modo, las que no se modifican tienen valores más altos en la columna kms, vamos a recuperar los valores unitarios de esa columna de las lineas que se habian modificado y que en su momento nos pasó desapercibido:

 'Kilómetros': array([ 350.,    6.,    1.,   10.,  300.,    0.,    5.,    7.,    2.,
        1000.,    9.,   15.,   25.,   30.,   38.,    3.,   12.,   50.,
         293.,   17.,    8.,    4.,   14.])

Vemos que todos los valores están entre 1 y 1000, será esta la condicion?, vamos a aplicarle el filtro al df original otra vez vs el filtrado a ver si llegamos al mismo numero de lineas, esa puede ser una comprobación fácil y rápida

In [ ]:
print(lineas_diff["Marca"].count()/2)
print(df_exer[(df_exer["Tipo de cambio"]=="Manual") &
              (df_exer["Tipo de combustible"]=="Híbrido") &
              (df_exer["Kilómetros"]<=1000)
             ]["Marca"].count())

96.0
96


#### ¡Eureka!

Ahora si hemos encontrado la respuesta a las dos primeras preguntas!,

¿Qué está sucendiendo? --> Está duplicando el valor del precio
¿Por qué está sucendiendo? --> Lo hace sólo en aquellas lineas de vehiculos manuales , hibridos y con menos de 1000 kms
¿Dónde está sucediendo? --> Ahora con esta información ya podemos debugar el código buscando donde se producen los cálculos de la columna precio para resolverlo!.

Es posible que el error sea de parametrización, que ya veremos en un proximo capítulo o por un filtro mal hecho, un cruce no deseado, una coma donde no debe... hay mil motivos, pero lo importante es que hayas entendido que para que un producto sea industrializable y funcional en una empresa, los valores deben ser validados a conciencia, y si falla en algún momento, debemos ser capaces de sumergirnos hasta el fondo para encontrar el origen del problema.

En nuestra experiencia profesional hemos visto decenas de programadores y data scientists que les gusta mucho programar pero luego no saben profundizar en los numeros para validar sus códigos... ERROR!, Un perfil completo debe ser capaz de resolverlo todo, de la A-z